In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv("train.csv")

In [3]:
train = train.drop('id', axis = 1)
train = train.drop('conversation_id', axis = 1)
train = train.drop('created_at', axis = 1)
train = train.drop('time', axis = 1)
train = train.drop('timezone', axis = 1)
train = train.drop('user_id', axis = 1)
train = train.drop('username', axis = 1)
train = train.drop('name', axis = 1)
train = train.drop('place', axis = 1)
train = train.drop('mentions', axis = 1)
train = train.drop('urls', axis = 1)
train = train.drop('photos', axis = 1)
train = train.drop('replies_count', axis = 1)
train = train.drop('retweets_count', axis = 1)
train = train.drop('likes_count', axis = 1)
train = train.drop('hashtags', axis = 1)
train = train.drop('cashtags', axis = 1)
train = train.drop('link', axis = 1)
train = train.drop('retweet', axis = 1)
train = train.drop('quote_url', axis = 1)
train = train.drop('video', axis = 1)
train = train.drop('near', axis = 1)
train = train.drop('geo', axis = 1)
train = train.drop('source', axis = 1)
train = train.drop('user_rt_id', axis = 1)
train = train.drop('user_rt', axis = 1)
train = train.drop('retweet_id', axis = 1)
train = train.drop('reply_to', axis = 1)
train = train.drop('retweet_date', axis = 1)
train = train.drop('translate', axis = 1)
train = train.drop('trans_src', axis = 1)
train = train.drop('trans_dest', axis = 1)

In [4]:
train.to_csv('train_tweets.csv', index = False)

In [5]:
training_tweets = 10820

In [6]:
test = pd.read_csv("test.csv")

In [7]:
test = test.drop('id', axis = 1)
test = test.drop('conversation_id', axis = 1)
test = test.drop('created_at', axis = 1)
test = test.drop('time', axis = 1)
test = test.drop('timezone', axis = 1)
test = test.drop('user_id', axis = 1)
test = test.drop('username', axis = 1)
test = test.drop('name', axis = 1)
test = test.drop('place', axis = 1)
test = test.drop('mentions', axis = 1)
test = test.drop('urls', axis = 1)
test = test.drop('photos', axis = 1)
test = test.drop('replies_count', axis = 1)
test = test.drop('retweets_count', axis = 1)
test = test.drop('likes_count', axis = 1)
test = test.drop('hashtags', axis = 1)
test = test.drop('cashtags', axis = 1)
test = test.drop('link', axis = 1)
test = test.drop('retweet', axis = 1)
test = test.drop('quote_url', axis = 1)
test = test.drop('video', axis = 1)
test = test.drop('near', axis = 1)
test = test.drop('geo', axis = 1)
test = test.drop('source', axis = 1)
test = test.drop('user_rt_id', axis = 1)
test = test.drop('user_rt', axis = 1)
test = test.drop('retweet_id', axis = 1)
test = test.drop('reply_to', axis = 1)
test = test.drop('retweet_date', axis = 1)
test = test.drop('translate', axis = 1)
test = test.drop('trans_src', axis = 1)
test = test.drop('trans_dest', axis = 1)

In [8]:
test.to_csv('test_tweets.csv', index = False)

In [9]:
testing_tweets = 609

Next step is to preprocess these stupid fucking tweets.

In [10]:
import preprocessor as p
from gensim.parsing.preprocessing import remove_stopwords

In [11]:
def preprocess_tweet(row):
    text = row['tweet']
    text = p.clean(text)
    return text

In [12]:
def stopword_removal(row):
    text = row['tweet']
    text = remove_stopwords(text)
    return text

In [13]:
train = train.dropna()
train = train.drop_duplicates()
train['tweet'] = train.apply(preprocess_tweet, axis=1)
train['tweet'] = train.apply(stopword_removal, axis=1)
train['tweet'] = train['tweet'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')

train.to_csv('train_processed.csv', index = False)

train.count()

date     10189
tweet    10189
dtype: int64

In [14]:
test = test.dropna()
test = test.drop_duplicates()
test['tweet'] = test.apply(preprocess_tweet, axis=1)
test['tweet'] = test.apply(stopword_removal, axis=1)
test['tweet'] = test['tweet'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')

test.to_csv('test_processed.csv', index = False)

test.count()

date     603
tweet    603
dtype: int64

Time to add some sentiment, come on down TextBlob

In [16]:
from textblob import TextBlob

In [17]:
polarity = lambda x: TextBlob(x).sentiment.polarity

train['polarity'] = train['tweet'].apply(polarity)

test['polarity'] = test['tweet'].apply(polarity)

In [18]:
train.to_csv('train_sentiment.csv', index = False)
test.to_csv('test_sentiment.csv', index = False)

Now we will drop the tweet column as it is only the date and the sentiment score which are of use to us here.

In [22]:
train = train.drop('tweet', axis=1)

KeyError: "['tweet'] not found in axis"

In [23]:
test = test.drop('tweet', axis = 1)

Now that we have the sentiment score for each day we can find the average

In [25]:
train['date'] = pd.to_datetime(train['date'], format="%Y/%m/%d")
train = train.groupby(train.date.dt.date).mean()

len(train)

1508

In [26]:
train.to_csv('train_average.csv', index = True)

In [27]:
test['date'] = pd.to_datetime(test['date'], format="%Y/%m/%d")
test = test.groupby(test.date.dt.date).mean()
len(test)

55

In [28]:
test.to_csv('test_average.csv', index = True)